[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/one-shot-talking-face/blob/main/one_shot_talking_face.ipynb)

In [ ]:
!git lfs install
%cd /content
!git clone https://huggingface.co/camenduru/pocketsphinx
%cd  /content/pocketsphinx
!sudo cmake --build build --target install

%cd /content
!git clone https://huggingface.co/camenduru/one-shot-talking-face
%cd /content/one-shot-talking-face
!pip install -r /content/one-shot-talking-face/requirements.txt
!chmod 755 /content/one-shot-talking-face/OpenFace/FeatureExtraction
!mkdir /content/out
!apt install libgtk2.0-0 jq -y
!pip install imageio-ffmpeg

In [ ]:
wav_file = "/content/audio.wav"  #@param {type:"string"}
image_file = "/content/image.jpg" #@param {type:"string"}

import os
os.environ['wav_file'] = wav_file
os.environ['image_file'] = image_file

!mkdir /content/train
!cp $wav_file /content/train/audio.wav
!cp $image_file /content/train/image.jpg

!pocketsphinx -phone_align yes single /content/train/audio.wav $text | jq '[.w[]|{word: (.t | ascii_upcase | sub("<S>"; "sil") | sub("<SIL>"; "sil") | sub("\\(2\\)"; "") | sub("\\(3\\)"; "") | sub("\\(4\\)"; "") | sub("\\[SPEECH\\]"; "SIL") | sub("\\[NOISE\\]"; "SIL")), phones: [.w[]|{ph: .t | sub("\\+SPN\\+"; "SIL") | sub("\\+NSN\\+"; "SIL"), bg: (.b*100)|floor, ed: (.b*100+.d*100)|floor}]}]' > /content/test.json
%cd /content/one-shot-talking-face
!python -B test_script.py --img_path /content/train/image.jpg --audio_path /content/train/audio.wav --phoneme_path /content/test.json --save_dir /content/out

In [ ]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 256):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video("/content/out/image_audio.mp4")